<font size="1">Copyright 2021, by the California Institute of Technology. ALL RIGHTS RESERVED. United States Government sponsorship acknowledged. Any commercial use must be negotiated with the Office of Technology Transfer at the California Institute of Technology.</font>
    
<font size="1">This software may be subject to U.S. export control laws and regulations. By accepting this document, the user agrees to comply with all applicable U.S. export laws and regulations. User has the responsibility to obtain export licenses, or other export authority as may be required, before exporting such information to foreign countries or providing access to foreign persons.<font>

## User-local Environment Creation
**This notebook must be run in the base kernel.**

This notebook provides the means by which additional conda environments that support specific science data packages are created local to the user (i.e.'user-local'). The implication of user-local installations is that they are retained across logins and server restarts (since the ~jovyan directory is retained and remounted each time). It also permits the user to customize the environment, again with those changes persisted across sessions.

This notebook provides the steps to create the following user-local environments:
- ISCE2/ISCE3
- MintPy/ARIATools
- PLAnT

Each environment also contains the PCM APIs (pele-client, otello, osaka) and tools (notebook-pge-wrapper).

#### Install kernda
Kernda will be used to make ipython aware of the environments we create.

In [ ]:
pip install kernda

### What environments are currently available?
Come back and re-execute this cell any time to see what has and hasn't been created. Those found under /home/jovyan are user-local while others are global.

In [ ]:
%conda env list

### Configure conda
This will make conda recognize directories under ~/.local/envs as their environment name. This only needs to be run once, ever.

In [ ]:
%%bash
mkdir -p ~/.local/envs
conda config --append envs_dirs ~/.local/envs

### ISCE2/ISCE3
This cell will create a user-local environment containing ISCE2 and ISCE3. This environment also contains the PCM APIs (pele, otello, osaka) and the notebook-pge-wrapper.

If some messages from tar ignoring unknown extended header keywords appear, they can be safely ignored.

In [ ]:
%%bash

env_base="/home/jovyan/.local/envs/isce"
python_version="3.9"

# (re)create the ISCE2/ISCE3 environment
mamba env create -f env_files/isce_conda_env.yml -p ${env_base} --force

# update the ipykernel to recognize the new kernel
conda run -n isce python -m ipykernel install --user --name isce --display-name "isce"
# now invoke kernda to modify the kernel file to ensure the kernel is invoked from within the conda environment
conda run -n isce kernda --env-dir ${env_base} --display-name isce -o ~/.local/share/jupyter/kernels/isce/kernel.json

isce_package=${env_base}/lib/python${python_version}/site-packages/isce

# patch isce2 to correct Land Processes DAAC URLs
pushd ${isce_package}
sed -i 's?http://e4ft?https://e4ft?g' `grep -rl "http://e4ft" ./`
popd

# install the license
tar zxf support_files/stanford_components.tgz -C ${isce_package}

# Additional ISCE set-up
source /opt/conda/bin/activate ${env_base}
conda env config vars set -n isce PATH=${env_base}/bin:${isce_package}/applications:${PATH}
conda env config vars set -n isce PYTHONPATH=${isce_package}:${PYTHONPATH}

### ISCE2/ISCE3 + RAiDER
This cell will create a user-local environment containing ISCE2, ISCE3 and RAiDER (Raytracing Atmospheric Delay Estimation for RADAR - https://github.com/dbekaert/RAiDER). This environment also contains the PCM APIs (pele, otello, osaka) and the notebook-pge-wrapper.

If some messages from tar ignoring unknown extended header keywords appear, they can be safely ignored.

In [ ]:
%%bash

env_base="/home/jovyan/.local/envs/isce_raider"
python_version="3.9"

# (re)create the ISCE2/ISCE3 environment that includes RAiDER
mamba env create -f env_files/isce_conda_env.yml -p ${env_base} --force
# for now, have to use a slightly customized raider dependency yml file which specifies the correct version of dem_stitcher
mamba env update -f env_files/environment_raider.yml -p ${env_base}

# install RAiDER, repo will be checked out into /tmp
pushd /tmp
if [ ! -d RAiDER ] && echo "Checking out RAiDER"
then
    git clone https://github.com/dbekaert/RAiDER
fi
cd RAiDER
conda run -n isce_raider python setup.py install

# update the ipykernel to recognize the new kernel
conda run -n isce_raider python -m ipykernel install --user --name isce_raider --display-name "isce_raider"
# now invoke kernda to modify the kernel file to ensure the kernel is invoked from within the conda environment
conda run -n isce_raider kernda --env-dir ${env_base} --display-name isce_raider -o ~/.local/share/jupyter/kernels/isce_raider/kernel.json

isce_package=${env_base}/lib/python${python_version}/site-packages/isce

# patch isce2 to correct Land Processes DAAC URLs
pushd ${isce_package}
sed -i 's?http://e4ft?https://e4ft?g' `grep -rl "http://e4ft" ./`
popd

# install the license
popd
tar zxf support_files/stanford_components.tgz -C ${isce_package}

# Additional ISCE set-up
source /opt/conda/bin/activate ${env_base}
conda env config vars set -n isce_raider PATH=${env_base}/bin:${isce_package}/applications:${PATH}
conda env config vars set -n isce_raider PYTHONPATH=${isce_package}:${PYTHONPATH}

### MintPy/ARIAtools
This cell will create a user-local environment containing MintPy and ARIAtools. This environment also contains the PCM APIs (pele, otello, osaka) and the notebook-pge-wrapper.

Note that the script below installs the default branch of ARIA-tools (currently 'dev'). If you'd like a specific branch (e.g. v1.1.2), change the git clone line to:

```git clone -b v1.1.2 https://github.com/aria-tools/ARIA-tools```

In [ ]:
%%bash

local_base="/home/jovyan/.local"
env_base=${local_base}/envs/mintpy

# (re)create the MintPy/ARIATools environment
mamba env create -f env_files/mintpy_conda_env.yml -p ${env_base} --force

# make the base mintpy files executable
chmod 755 ${env_base}/lib/python*/site-packages/mintpy/*.py

# install/build ARIA-tools
aria_install_dir=${local_base}/tools/ARIA-tools
if [ ! -d ${aria_install_dir} ] && echo "Checking out ARIA-tools"
then
    # add -b <branch/tag> to the following command to install one other than dev
    git clone https://github.com/aria-tools/ARIA-tools ${aria_install_dir}
fi

# even if the directory previously exists, this has to be done since the environment is being recreated
pushd ${aria_install_dir}
git pull
mamba run -n mintpy python setup.py build
mamba run -n mintpy python setup.py install
popd

# update the ipykernel to recognize the new kernel
conda run -n mintpy python -m ipykernel install --user --name mintpy --display-name "mintpy"
# now invoke kernda to modify the kernel file to ensure the kernel is invoked from within the conda environment
conda run -n mintpy kernda --env-dir ${env_base} --display-name mintpy -o ~/.local/share/jupyter/kernels/mintpy/kernel.json

# Additional ARIA-tools set-up
source /opt/conda/bin/activate ${env_base}
conda env config vars set -n mintpy PATH=${aria_install_dir}:${PATH}
conda env config vars set -n mintpy PYTHONPATH=${aria_install_dir}:${PYTHONPATH}

### MintPy from insarlab repository and ARIAtools
This cell will create a user-local environment containing MintPy as checked out from insarlab's github repo which allows access to newer features/fixes not yet available through conda-forge. ARIAtools is installed from github as in the above cell (see the note about installing different branches). This environment also contains the PCM APIs (pele, otello, osaka) and the notebook-pge-wrapper.

In [ ]:
%%bash

local_base="/home/jovyan/.local"
env_base=${local_base}/envs/mintpy

# (re)create the MintPy/ARIATools environment
mamba env create -f env_files/mintpy_conda_env_base.yml -p ${env_base} --force

mintpy_install_dir=${env_base}/tools/MintPy
if [ ! -d ${mintpy_install_dir} ] && echo "Checking out MintPy"
then
   git clone https://github.com/insarlab/MintPy.git ${mintpy_install_dir}
fi

source /opt/conda/bin/activate ${env_base}
sudo yum install -y gcc-c++
pip install kernda

# even if the directory previously exists, this has to be done since the environment is being recreated
pushd ${mintpy_install_dir}
git pull
python -m pip install .
popd

# install/build ARIA-tools
aria_install_dir=${env_base}/tools/ARIA-tools
if [ ! -d ${aria_install_dir} ] && echo "Checking out ARIA-tools"
then
    # add -b <branch/tag> to the following command to install one other than dev
    git clone https://github.com/aria-tools/ARIA-tools ${aria_install_dir}
fi

# even if the directory previously exists, this has to be done since the environment is being recreated
pushd ${aria_install_dir}
git pull
conda run -n mintpy python setup.py build
conda run -n mintpy python setup.py install
popd

# update the ipykernel to recognize the new kernel
conda run -n mintpy python -m ipykernel install --user --name mintpy --display-name "mintpy"
# now invoke kernda to modify the kernel file to ensure the kernel is invoked from within the conda environment
conda run -n mintpy kernda --env-dir ${env_base} --display-name mintpy -o ~/.local/share/jupyter/kernels/mintpy/kernel.json

# Additional ARIA-tools set-up
conda env config vars set -n mintpy PATH=${mintpy_install_dir}/mintpy:${aria_install_dir}:${PATH}
conda env config vars set -n mintpy PYTHONPATH=${mintpy_install_dir}:${aria_install_dir}:${PYTHONPATH}

### PLAnT
This cell will create a user-local environment containing PLAnT as well as the aforementioned PCM APIs and notebook-pge-wrapper. This environment is very straight-forward to create.

In [ ]:
%%bash

env_base="/home/jovyan/.local/envs/plant"

# (re)create the PLAnT environment
mamba env create -f env_files/plant_conda_env.yml -p ${env_base} --force

# update the ipykernel to recognize the new kernel
conda run -n plant python -m ipykernel install --user --name plant --display-name "plant"
# now invoke kernda to modify the kernel file to ensure the kernel is invoked from within the conda environment
conda run -n plant kernda --env-dir ${env_base} --display-name plant -o ~/.local/share/jupyter/kernels/plant/kernel.json

### TrackFrameDB
This cell will create a user-local environment for running the track frame database notebook and scripts.
This environment is very straight-forward to create.

In [5]:
%%bash

env_name="track_frame_db"
env_base="/home/jovyan/.local/envs/${env_name}"

# (re)create the Track Frame DB environment
mamba env create -f env_files/${env_name}_conda_env.yml -p ${env_base} --force

# update the ipykernel to recognize the new kernel
mamba run -n ${env_name} python -m ipykernel install --user --name ${env_name} --display-name "${env_name}"
# now invoke kernda to modify the kernel file to ensure the kernel is invoked from within the conda environment
mamba run -n ${env_name} kernda --env-dir ${env_base} --display-name ${env_name} -o ~/.local/share/jupyter/kernels/${env_name}/kernel.json



Looking for: ['gdal', 'h5py', 'numpy', 'geopandas', 'jupyterlab', 'pandas', 'matplotlib', 'fiona']


Transaction

  Prefix: /home/jovyan/.local/envs/track_frame_db

  Updating specs:

   - gdal
   - h5py
   - numpy
   - geopandas
   - jupyterlab
   - pandas
   - matplotlib
   - fiona


  Package                               Version  Build                    Channel                    Size
───────────────────────────────────────────────────────────────────────────────────────────────────────────
  Install:
───────────────────────────────────────────────────────────────────────────────────────────────────────────

  + _libgcc_mutex                           0.1  conda_forge              conda-forge/linux-64     Cached
  + _openmp_mutex                           4.5  2_gnu                    conda-forge/linux-64     Cached
  + alsa-lib                              1.2.8  h166bdaf_0               conda-forge/linux-64     Cached
  + anyio                                 3.7.0  pyhd8ed1ab

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.



Installed kernelspec track_frame_db in /home/jovyan/.local/share/jupyter/kernels/track_frame_db



Wrote to /home/jovyan/.local/share/jupyter/kernels/track_frame_db/kernel.json



{
  "argv": [
    "bash",
    "-c",
    "source \"/opt/conda/bin/activate\" \"/home/jovyan/.local/envs/track_frame_db\" && exec /home/jovyan/.local/envs/track_frame_db/bin/python -m ipykernel_launcher -f '{connection_file}' "
  ],
  "display_name": "track_frame_db",
  "language": "python",
  "metadata": {
    "debugger": true
  }
}



### Creating other user-local environments.

As shown above, the command used to create a conda environment is:

```conda env create <arguments>```

In the above cases, the ```-f``` switch is used to denote the input file which describes the environment's contents, including the source conda channels (e.g. default, conda-forge) and the environment prefix (in this case, subdirectories of /home/jovyan/.local/envs). The prefix is responsible for designating where the environment's contents will reside and if a subdirectory of the user (i.e. jovyan), the environment is user-local. Since the ~jovyan directory is persisted and remounted for each jupyter login, these environments also persist and do not have to be created each time you log into jupyter.

The --force flag in the command lines used above forces conda to recreate (rather than just update) the environment from scratch.

For clarity and organization, it's best to maintain the local environments in the same user subdirectory.

To create a new user-local environment from scratch, use the following command:

```conda env create -p /home/jovyan/.local/envs/<env-name>```


To start with a copy of another environment (such as pcm_tools), omit the ```env``` and use the ```--clone``` command line option:

```conda create -p /home/jovyan/.local/envs/<env-name> --clone pcm_tools```

Once created, switch to the new environment accordingly (this is only applicable in terminal windows):

```conda activate <env-name>```

Currently active environments and their prefix can be listed by:

```conda env list```

An environment can be removed by:

```conda env remove -n <env name>``` - to remove by name

or

```conda env remove -p <prefix>``` - to remove by prefix

Finally, to make ipython aware of the environment, invoke ipykernel:

```conda run -n <env name> python -m ipykernel install --user --name <env name> --display-name <env name>```

then run kernda:

```conda run -n <env name> kernda --env-dir ~/.local/share/<env name> --display-name <env name> \
         -o ~/.local/share/jupyter/kernels/<env name>/kernel.json```

Running kernda ensures that the kernel is launched from within the associated conda environment. This means that any shell commands invoked within the notebook (e.g. using the %%bash magic) execute within that environment.

You will need to reload the jupyter landing page in order to see the environment under the ```New``` pull down.

<font size="1">This notebook is compatible with NISAR Jupyter Server Image v1.7.3 and above</font>